# Topic Modeling

In [1]:
# In this workshop we perform topic modeling using gensim

# We are using the subnews dataset that we used last week. 
# The "Class" labels here are only used for sanity check of the topics discovered later.
# Remember, in actual use of topic modelling, the documents DON'T come with labeled classes.
# It's unsupervised learning.

import pandas as pd
news=pd.read_table('r8-train-all-terms.txt',header=None,names = ["Class", "Text"])
subnews=news[(news.Class=="trade")| (news.Class=='crude')|(news.Class=='money-fx') ]
subnews.head()

,Class,Text
15,trade,brazil anti inflation plan limps to anniversar...
43,crude,diamond shamrock dia cuts crude prices diamond...
55,crude,opec may have to meet to firm prices analysts ...
76,crude,texaco canada cuts crude prices canadian cts b...
77,crude,texaco canada txc lowers crude postings texaco...


In [2]:
# Let's use the similar preprocessing we used last week.
# The output of each document is a list of tokens.

import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)

In [3]:
#split the data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subnews.Text, subnews.Class, test_size=0.33, random_state=12)


In [4]:
# Apply preprocessing to every document in the training set.
toks_train = X_train.apply(pre_process)


In [5]:
!pip install gensim

  Running setup.py bdist_wheel for smart-open: started
  Running setup.py bdist_wheel for smart-open: finished with status 'done'
  Stored in directory: C:\Users\e0267553\AppData\Local\pip\Cache\wheels\73\f1\9b\ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file: started
  Running setup.py bdist_wheel for bz2file: finished with status 'done'
  Stored in directory: C:\Users\e0267553\AppData\Local\pip\Cache\wheels\81\75\d6\e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


Exception:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path,
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_set.py", line 784, in install
    **kwargs
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_install.py", line 851, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\req\req_install.py", line 1064, in move_wheel_files
    isolated=self.isolated,
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\wheel.py", line 345, in move_wheel_files
    clobber(source, lib_dir, True)
  File "C:\Program Files\Anaconda3\lib\site-packages\pip\wheel.py", line 323, in clobber
    shutil.copyfile(srcfile, destfile)
  File "C:\Program File

In [7]:
import logging
import gensim 
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
# Prepare a vocabulary dictionary.
dictionary = corpora.Dictionary(toks_train)
print(dictionary)

2018-09-15 15:01:29,763 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-15 15:01:29,812 : INFO : built Dictionary(5971 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...) from 475 documents (total 57987 corpus positions)


Dictionary(5971 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


In [9]:
# Filter off any words with document frequency less than 3, or appearing in more than 80% documents
dictionary.filter_extremes(no_below=3, no_above=0.7)
print(dictionary)

2018-09-15 15:01:31,962 : INFO : discarding 3537 tokens: [('frightening', 2), ('mandating', 2), ('reuter', 408), ('said', 403), ('toughening', 2), ('wedge', 2), ('aggravated', 1), ('delay', 1), ('devaluation', 2), ('herald', 1)]...
2018-09-15 15:01:31,963 : INFO : keeping 2434 tokens which were in no less than 3 and no more than 332 (=70.0%) documents
2018-09-15 15:01:31,966 : INFO : resulting dictionary: Dictionary(2434 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


Dictionary(2434 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


In [10]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [11]:
# Use the dictionary to prepare a DTM (using TF)
dtm_train = [dictionary.doc2bow(d) for d in toks_train ]

In [12]:
# Build an LDA model for 3 topics out of the DTM
%time lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics = 3,  alpha='auto',chunksize=30,id2word = dictionary, passes = 20,random_state=432)
#chunksize (int, optional) – Number of documents to be used in each training chunk.
#passes (int, optional) – Number of passes through the corpus during training.

2018-09-15 15:02:18,783 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2018-09-15 15:02:18,784 : INFO : using symmetric eta at 0.3333333333333333
2018-09-15 15:02:18,785 : INFO : using serial LDA version on this node
2018-09-15 15:02:18,787 : INFO : running online (multi-pass) LDA training, 3 topics, 20 passes over the supplied corpus of 475 documents, updating model once every 30 documents, evaluating perplexity every 300 documents, iterating 50x with a convergence threshold of 0.001000
2018-09-15 15:02:18,788 : INFO : PROGRESS: pass 0, at document #30/475
2018-09-15 15:02:19,002 : INFO : optimized alpha [0.52708918, 0.50643796, 0.48903793]
2018-09-15 15:02:19,003 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:19,005 : INFO : topic #0 (0.527): 0.027*"trade" + 0.013*"china" + 0.013*"oil" + 0.012*"year" + 0.011*"export" + 0.011*"would" + 0.009*"last" + 0.009*"gatt" + 0.008*"system" + 0.008*"currency"
2018-09-

2018-09-15 15:02:19,190 : INFO : optimized alpha [0.57153779, 0.47655696, 0.52661896]
2018-09-15 15:02:19,191 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:19,193 : INFO : topic #0 (0.572): 0.028*"trade" + 0.017*"japan" + 0.015*"would" + 0.009*"year" + 0.008*"world" + 0.008*"agreement" + 0.007*"minister" + 0.007*"import" + 0.007*"foreign" + 0.007*"market"
2018-09-15 15:02:19,194 : INFO : topic #1 (0.477): 0.046*"oil" + 0.029*"mln" + 0.018*"price" + 0.014*"barrel" + 0.014*"opec" + 0.014*"pct" + 0.013*"market" + 0.012*"production" + 0.010*"stg" + 0.010*"bank"
2018-09-15 15:02:19,195 : INFO : topic #2 (0.527): 0.020*"billion" + 0.015*"dlrs" + 0.015*"dollar" + 0.015*"bank" + 0.014*"japan" + 0.014*"official" + 0.014*"japanese" + 0.011*"market" + 0.009*"trade" + 0.009*"yen"
2018-09-15 15:02:19,196 : INFO : topic diff=0.477447, rho=0.333333
2018-09-15 15:02:19,220 : INFO : -7.282 per-word bound, 155.7 perplexity estimate based on a held-out corpus o

2018-09-15 15:02:19,399 : INFO : topic #1 (0.398): 0.043*"oil" + 0.025*"mln" + 0.022*"price" + 0.018*"opec" + 0.017*"bpd" + 0.016*"crude" + 0.012*"dlrs" + 0.012*"barrel" + 0.011*"pct" + 0.009*"market"
2018-09-15 15:02:19,400 : INFO : topic #2 (0.366): 0.028*"billion" + 0.018*"dlrs" + 0.018*"bank" + 0.015*"official" + 0.013*"japanese" + 0.011*"dollar" + 0.010*"market" + 0.010*"february" + 0.009*"year" + 0.008*"last"
2018-09-15 15:02:19,401 : INFO : topic diff=0.275793, rho=0.236801
2018-09-15 15:02:19,402 : INFO : PROGRESS: pass 1, at document #60/475
2018-09-15 15:02:19,413 : INFO : optimized alpha [0.42603716, 0.34051889, 0.33244377]
2018-09-15 15:02:19,414 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:19,416 : INFO : topic #0 (0.426): 0.033*"trade" + 0.013*"would" + 0.012*"country" + 0.011*"japan" + 0.008*"year" + 0.007*"state" + 0.007*"foreign" + 0.007*"agreement" + 0.006*"economic" + 0.006*"market"
2018-09-15 15:02:19,416 : INFO : topic #

2018-09-15 15:02:19,565 : INFO : PROGRESS: pass 1, at document #300/475
2018-09-15 15:02:19,578 : INFO : optimized alpha [0.33410797, 0.27763724, 0.29562563]
2018-09-15 15:02:19,579 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:19,581 : INFO : topic #0 (0.334): 0.031*"trade" + 0.016*"japan" + 0.016*"would" + 0.009*"world" + 0.008*"year" + 0.008*"agreement" + 0.008*"country" + 0.007*"minister" + 0.007*"market" + 0.006*"foreign"
2018-09-15 15:02:19,581 : INFO : topic #1 (0.278): 0.048*"oil" + 0.026*"mln" + 0.020*"price" + 0.014*"opec" + 0.014*"dlrs" + 0.013*"barrel" + 0.013*"pct" + 0.011*"crude" + 0.011*"market" + 0.010*"stg"
2018-09-15 15:02:19,582 : INFO : topic #2 (0.296): 0.024*"billion" + 0.020*"bank" + 0.017*"dollar" + 0.016*"dlrs" + 0.014*"japanese" + 0.013*"official" + 0.012*"market" + 0.011*"japan" + 0.010*"trade" + 0.009*"chip"
2018-09-15 15:02:19,583 : INFO : topic diff=0.243499, rho=0.236801
2018-09-15 15:02:19,584 : INFO : PROGRESS

2018-09-15 15:02:19,741 : INFO : topic #0 (0.297): 0.032*"trade" + 0.013*"would" + 0.012*"country" + 0.012*"japan" + 0.008*"year" + 0.007*"state" + 0.006*"agreement" + 0.006*"market" + 0.006*"world" + 0.006*"united"
2018-09-15 15:02:19,743 : INFO : topic #1 (0.230): 0.043*"oil" + 0.025*"mln" + 0.022*"price" + 0.019*"opec" + 0.017*"bpd" + 0.016*"crude" + 0.014*"dlrs" + 0.013*"barrel" + 0.009*"ecuador" + 0.009*"saudi"
2018-09-15 15:02:19,744 : INFO : topic #2 (0.226): 0.030*"billion" + 0.024*"bank" + 0.019*"dollar" + 0.016*"dlrs" + 0.014*"official" + 0.012*"japanese" + 0.010*"february" + 0.010*"market" + 0.010*"january" + 0.009*"trade"
2018-09-15 15:02:19,744 : INFO : topic diff=0.258099, rho=0.230429
2018-09-15 15:02:19,745 : INFO : PROGRESS: pass 2, at document #90/475
2018-09-15 15:02:19,759 : INFO : optimized alpha [0.28034431, 0.2376004, 0.21250975]
2018-09-15 15:02:19,760 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:19,761 : INFO : topic

...

2018-09-15 15:02:24,353 : INFO : topic diff=0.215305, rho=0.171920
2018-09-15 15:02:24,367 : INFO : -6.831 per-word bound, 113.8 perplexity estimate based on a held-out corpus of 25 documents with 2811 words
2018-09-15 15:02:24,369 : INFO : PROGRESS: pass 17, at document #475/475
2018-09-15 15:02:24,376 : INFO : optimized alpha [0.19922766, 0.12821767, 0.15752882]
2018-09-15 15:02:24,377 : INFO : merging changes from 25 documents into a model of 475 documents
2018-09-15 15:02:24,378 : INFO : topic #0 (0.199): 0.033*"trade" + 0.014*"japan" + 0.014*"would" + 0.010*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"official" + 0.007*"agreement" + 0.007*"market"
2018-09-15 15:02:24,379 : INFO : topic #1 (0.128): 0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct"
2018-09-15 15:02:24,381 : INFO : topic #2 (0.158): 0.032*"bank" + 0.026*"billion" + 0.018*"mln" + 0.017*"market" 

2018-09-15 15:02:24,501 : INFO : topic #1 (0.130): 0.048*"oil" + 0.025*"price" + 0.019*"mln" + 0.018*"opec" + 0.016*"dlrs" + 0.014*"crude" + 0.014*"barrel" + 0.013*"bpd" + 0.010*"saudi" + 0.010*"production"
2018-09-15 15:02:24,502 : INFO : topic #2 (0.163): 0.029*"bank" + 0.027*"billion" + 0.018*"mln" + 0.018*"dollar" + 0.016*"market" + 0.015*"rate" + 0.015*"exchange" + 0.014*"currency" + 0.011*"pct" + 0.011*"dlrs"
2018-09-15 15:02:24,502 : INFO : topic diff=0.166060, rho=0.169435
2018-09-15 15:02:24,503 : INFO : PROGRESS: pass 18, at document #270/475
2018-09-15 15:02:24,513 : INFO : optimized alpha [0.2010857, 0.13029647, 0.15958515]
2018-09-15 15:02:24,514 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:24,516 : INFO : topic #0 (0.201): 0.029*"trade" + 0.019*"japan" + 0.014*"would" + 0.011*"japanese" + 0.009*"official" + 0.009*"country" + 0.009*"year" + 0.008*"market" + 0.007*"minister" + 0.007*"agreement"
2018-09-15 15:02:24,517 : INFO : to

2018-09-15 15:02:24,656 : INFO : topic diff=0.172029, rho=0.169435
2018-09-15 15:02:24,657 : INFO : PROGRESS: pass 19, at document #30/475
2018-09-15 15:02:24,666 : INFO : optimized alpha [0.19981769, 0.13043033, 0.15349902]
2018-09-15 15:02:24,667 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:24,669 : INFO : topic #0 (0.200): 0.034*"trade" + 0.013*"japan" + 0.013*"would" + 0.011*"country" + 0.009*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"agreement" + 0.007*"official" + 0.007*"market"
2018-09-15 15:02:24,670 : INFO : topic #1 (0.130): 0.046*"oil" + 0.025*"price" + 0.018*"opec" + 0.017*"dlrs" + 0.017*"crude" + 0.017*"bpd" + 0.017*"mln" + 0.013*"barrel" + 0.009*"company" + 0.009*"pct"
2018-09-15 15:02:24,671 : INFO : topic #2 (0.153): 0.031*"bank" + 0.027*"billion" + 0.020*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"currency" + 0.013*"rate" + 0.013*"exchange" + 0.012*"stg" + 0.011*"pct"
2018-09-15 15:02:24,671 : INFO : topic diff=

2018-09-15 15:02:24,794 : INFO : topic #1 (0.129): 0.050*"oil" + 0.025*"price" + 0.018*"mln" + 0.018*"opec" + 0.017*"dlrs" + 0.015*"barrel" + 0.014*"crude" + 0.011*"pct" + 0.011*"bpd" + 0.011*"saudi"
2018-09-15 15:02:24,795 : INFO : topic #2 (0.158): 0.031*"bank" + 0.026*"billion" + 0.021*"dollar" + 0.019*"mln" + 0.016*"market" + 0.015*"currency" + 0.013*"rate" + 0.012*"exchange" + 0.012*"pct" + 0.011*"stg"
2018-09-15 15:02:24,796 : INFO : topic diff=0.178439, rho=0.167054
2018-09-15 15:02:24,811 : INFO : -7.042 per-word bound, 131.8 perplexity estimate based on a held-out corpus of 30 documents with 2726 words
2018-09-15 15:02:24,812 : INFO : PROGRESS: pass 19, at document #300/475
2018-09-15 15:02:24,820 : INFO : optimized alpha [0.19917622, 0.13374747, 0.16132151]
2018-09-15 15:02:24,821 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 15:02:24,822 : INFO : topic #0 (0.199): 0.031*"trade" + 0.019*"japan" + 0.014*"would" + 0.011*"japanese" + 0.009*"

Wall time: 6.14 s


In [13]:
# To see the topics, with the most probable words in each topic. What topics to you see? 
lda.show_topics()

[(0,
  '0.033*"trade" + 0.014*"japan" + 0.014*"would" + 0.010*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"official" + 0.007*"agreement" + 0.007*"market"'),
 (1,
  '0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct"'),
 (2,
  '0.032*"bank" + 0.026*"billion" + 0.019*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"rate" + 0.013*"currency" + 0.013*"exchange" + 0.011*"pct" + 0.010*"stg"')]

In [14]:
# You can also request to see more words per topic
lda.show_topics(num_words=20)

[(0,
  '0.033*"trade" + 0.014*"japan" + 0.014*"would" + 0.010*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"official" + 0.007*"agreement" + 0.007*"market" + 0.007*"world" + 0.006*"foreign" + 0.006*"united" + 0.006*"minister" + 0.006*"new" + 0.006*"last" + 0.005*"told" + 0.005*"import" + 0.005*"industry" + 0.005*"bill"'),
 (1,
  '0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct" + 0.008*"production" + 0.008*"last" + 0.007*"output" + 0.007*"petroleum" + 0.007*"ecuador" + 0.006*"day" + 0.006*"saudi" + 0.006*"would" + 0.006*"year" + 0.006*"per"'),
 (2,
  '0.032*"bank" + 0.026*"billion" + 0.019*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"rate" + 0.013*"currency" + 0.013*"exchange" + 0.011*"pct" + 0.010*"stg" + 0.010*"dlrs" + 0.008*"money" + 0.008*"central" + 0.008*"deficit" + 0.008*"paris" + 0.007*"system" + 0.007*"monetary" + 0.007*"year" + 0.006*"treasury" + 0.0

In [15]:
# Get the topic distribution of documents
dtopics_train = lda.get_document_topics(dtm_train)

In [16]:
# Get the topic likelihood for the first document in train set
for i in range(0, 5):
    print(dtopics_train[i])

[(0, 0.99748653)]
[(0, 0.09187004), (2, 0.9053942)]
[(0, 0.75051743), (1, 0.11056269), (2, 0.13891992)]
[(0, 0.96214074), (1, 0.016980482), (2, 0.020878794)]
[(1, 0.96449941), (2, 0.03264432)]


In [17]:
# Pick the topic with the highest probability for each document, map it to the label
# NOTE: the mapping may change in a different run
from operator import itemgetter
top_train = [ max(t, key=itemgetter(1))[0] for t in dtopics_train ]
dict = {1: 'crude', 2: 'money-fx', 0: 'trade'}
topic_train = [ dict[t] for t in top_train ]

In [18]:
# Now let's see how well these topics match the actual categories
import numpy as np
from sklearn import metrics
print(metrics.confusion_matrix(topic_train, y_train))
print(np.mean(topic_train == y_train) )
print(metrics.classification_report(topic_train, y_train))

[[153   1   1]
 [  3 107  33]
 [ 18  18 141]]
0.844210526316
             precision    recall  f1-score   support

      crude       0.88      0.99      0.93       155
   money-fx       0.85      0.75      0.80       143
      trade       0.81      0.80      0.80       177

avg / total       0.84      0.84      0.84       475



In [19]:
# The typical practice is to use the reserved test set for evaluation
toks_test = X_test.apply(pre_process)
dtm_test = [dictionary.doc2bow(d) for d in toks_test ]
dtopics_test = lda.get_document_topics(dtm_test)
top_test = [ max(t,key=itemgetter(1))[0] for t in dtopics_test ]
topic_test = [ dict[t] for t in top_test ]
print(metrics.confusion_matrix(topic_test, y_test))
print(np.mean(topic_test == y_test) )
print(metrics.classification_report(topic_test, y_test))

[[62  1  0]
 [ 4 66 16]
 [13 13 60]]
0.8
             precision    recall  f1-score   support

      crude       0.78      0.98      0.87        63
   money-fx       0.82      0.77      0.80        86
      trade       0.79      0.70      0.74        86

avg / total       0.80      0.80      0.80       235

